# Trabalho Final - Rede Aberta de Filas
## Modelagem de Avaliação e Desempenho 2024.2
### Alunas: Melissa Pereira Guarilha, Mellanie Pereira Guarilha

O objetivo do projeto foi simular uma rede aberta de filas com três servidores (S1, S2 e S3) para avaliar experimentalmente métricas relacionadas ao tempo de processamento de jobs no sistema. Foram analisadas três situações distintas, variando as distribuições de tempo de serviço em cada servidor, com o intuito de calcular o tempo médio e o desvio padrão do tempo de permanência dos jobs no sistema.

## Simulação

A solução idealizada para a simulação foi desenvolver um esquema com dois tipos distintos de servidores a serem instanciados: servidores de entrada e servidores de saída. Cada servidor é responsável por processar um job durante um intervalo de tempo definido.

Os jobs são inicialmente criados seguindo uma taxa de chegada que obedece a uma distribuição exponencial (variável aleatória) com média de 0,5. Após a criação, os jobs, juntamente com seus tempos de chegada, são enfileirados no servidor 1. Em seguida, os jobs são processados de acordo com a disponibilidade do servidor 1, respeitando a regra de que só podem ser atendidos após o momento de sua criação (ou seja, o tempo de chegada deve ser maior ou igual ao tempo atual). Após serem processados no servidor 1, os jobs são encaminhados para seleção entre os servidores de saída 2 e 3.

Optou-se por adicionar todos os jobs à fila imediatamente após sua criação para simplificar o processo de geração e organização dos jobs, permitindo uma simulação mais direta do sistema e a avaliação de seu tempo médio.

As distribuições de tempo de serviço foram configuradas de maneira distinta para cada tipo de servidor, com funções específicas para refletir as características desejadas de cada um.

In [2]:
from typing import Callable, Tuple
import numpy as np
import queue
import random

In [3]:
class Job:
  """
  Define um job.

  Attributes
    ----------
    id : int
        O id do job.
    tempo_de_chegada : int
        O tempo de chegada do job no sistema.
    tempo_de_saida : int
        O tempo de saida do job no sistema.
  """
  def __init__(self, id: int, tempo_de_chegada: float):
    self.id = id
    self.tempo_de_chegada = tempo_de_chegada
    self.tempo_de_saida = 0

In [ ]:
def gera_jobs(quantidade_de_jobs: int, warmup: int, tempo_entre_chegadas: float = 0.5) -> list[Job]:
  """
  Gera uma lista de objetos Job com diferentes tempos de chegada.
  Os tempos de chegada seguem uma V.A Exponencial de média 0.5.
  """
  tempo_atual = 0
  jobs = []
  for i in range(0, quantidade_de_jobs + warmup):
    jobs.append(Job(i, tempo_atual))
    tempo_atual += np.random.exponential(tempo_entre_chegadas)
  return jobs

In [ ]:
class ServidorInicial():
  def __init__(self, tempo_de_serviço: float):
    self.tempo_de_serviço = tempo_de_serviço
    self.ocupado = False
    self.tempo_previsto_de_termino = 0
    self.fila = queue.Queue()
    self.job_trabalhando = None
  
  def trabalha(self, tempo_atual: float, job: Job = None):
    """
    Se o servidor não estiver ocupado, começa a servir um job da fila.
    """
    if not self.ocupado and not self.fila.empty():

      # condição para não rodar jobs que teoricamente "ainda não chegaram"
      if tempo_atual < self.fila.queue[0].tempo_de_chegada:
        return

      # pega um job da fila
      job = self.fila.get(job)
      self.job_trabalhando = job
      job.tempo_de_saida = tempo_atual + self.tempo_de_serviço

      self.ocupado = True

  def conclui_trabalho(self, tempo_atual: float) -> Job:
    """
    Conclui o trabalho que o servidor estava trabalhando.
    """
    if self.ocupado and tempo_atual >= self.job_trabalhando.tempo_de_saida:
      job_saindo = self.job_trabalhando

      self.ocupado = False
      self.job_trabalhando = None

      return job_saindo

In [ ]:
class ServidoresFinais():
  def __init__(self, tempo_de_serviço: float, chance_de_retornar: float = 0):
    self.tempo_de_serviço = tempo_de_serviço
    self.chance_de_retornar = chance_de_retornar
    self.ocupado = False
    self.tempo_previsto_de_termino = 0
    self.fila = queue.Queue()
  
  def trabalha(self, tempo_atual: float):
    """
    Se o servidor não estiver ocupado, começa a servir um job da fila.
    """
    job = None

    if not self.ocupado and not self.fila.empty():
      job = self.fila.get(job)
      job.tempo_de_saida = tempo_atual + self.tempo_de_serviço

      self.ocupado = True
      self.tempo_previsto_de_termino = tempo_atual + self.tempo_de_serviço

      # calcula se o job retornará para fila com probabilidade 'chance_de_retornar'
      if random.random() < self.chance_de_retornar:
        self.fila.put(job)
  
  def conclui_trabalho(self, tempo_atual: float):
    """
    Conclui o trabalho que o servidor estava trabalhando e serve o próximo job que está na fila.
    """
    if self.ocupado and tempo_atual >= self.tempo_previsto_de_termino:
      self.ocupado = False
      self.tempo_previsto_de_termino = 0

      self.trabalha(tempo_atual)

In [ ]:
def sistema(jobs: list[Job], funcao_servidor_1: Callable[[], float], funcao_servidor_2: Callable[[], float], funcao_servidor_3: Callable[[], float]):
  """
  Simula um sistema com três servidores.
  """
  tempo_atual = 0

  Servidor1 = ServidorInicial(funcao_servidor_1())
  Servidor2 = ServidoresFinais(funcao_servidor_2(), 0.2)
  Servidor3 = ServidoresFinais(funcao_servidor_3())

  # opções e probabilidades dos servidores 2 e 3
  opcoes = ["S2", "S3"]
  probabilidades = [0.5, 0.5]

  # insere os jobs na fila do servidor 1
  for job in jobs:
    Servidor1.fila.put(job)

  # o primeiro job a chegar é sempre executado direto, dado que a fila está vazia
  Servidor1.trabalha(tempo_atual)

  while not Servidor1.fila.empty() or Servidor1.ocupado or not Servidor2.fila.empty() or Servidor2.ocupado or not Servidor3.fila.empty() or Servidor3.ocupado:

    # se job em execução tiver terminado o trabalho, libera o servidor
    job = Servidor1.conclui_trabalho(tempo_atual)
    # tenta executar o próximo serviço se o servidor estiver livre
    Servidor1.trabalha(tempo_atual)
    
    # se algum job saiu do servidor 1
    if job:
      # escolhe o servidor 2 ou 3 com base nas probabilidades
      servidor_escolhido = random.choices(opcoes, probabilidades)[0]

      if servidor_escolhido == "S2":
        # adiciona o job na fila do servidor 2
        Servidor2.fila.put(job)
        Servidor2.trabalha(tempo_atual)
      else:
        # adiciona o job na fila do servidor 3
        Servidor3.fila.put(job)
        Servidor3.trabalha(tempo_atual)

    # se job em execução tiver terminado o trabalho, libera os servidores e inicia o próximo job que está na fila 
    Servidor2.conclui_trabalho(tempo_atual)
    Servidor3.conclui_trabalho(tempo_atual)

    tempo_atual += 0.01

In [ ]:
# Situação 1: os tempos de serviço são fixos, determinísticos, e iguais a 0.4s, 0.6s e 0.95s, respectivamente.
def S1_deterministico(): return 0.4
def S2_deterministico(): return 0.6
def S3_deterministico(): return 0.95

# Situação 2: os tempos de serviço nos três servidores são V.A.s uniformes nos intervalos (0.1, 0.7), (0.1, 1.1) e (0.1, 1.8), respectivamente. 
def S1_uniforme(): return np.random.uniform(0.1, 0.7)
def S2_uniforme(): return np.random.uniform(0.1, 1.1)
def S3_uniforme(): return np.random.uniform(0.1, 1.8)

# Situação 3: os tempos de serviço são V.A.s exponenciais com médias 0.4s, 0.6s e 0.95s, respectivamente.
def S1_exponencial(): return np.random.exponential(0.4)
def S2_exponencial(): return np.random.exponential(0.6)
def S3_exponencial(): return np.random.exponential(0.95)

### Execução da simulação

A simulação foi realizada utilizando 100.000 jobs de aquecimento (warmup) e 100.000 jobs "reais" em cada cenário analisado. Inicialmente, a ideia era utilizar apenas 10.000 jobs, mas, devido à falta de convergência dos valores obtidos, o número foi ampliado para 100.000.

In [202]:
QUANTIDADE_DE_JOBS, WARMUP = 100000, 100000

In [203]:
jobs_deterministicos = gera_jobs(QUANTIDADE_DE_JOBS, WARMUP)
sistema(jobs_deterministicos, S1_deterministico, S2_deterministico, S3_deterministico)

In [204]:
jobs_uniformes = gera_jobs(QUANTIDADE_DE_JOBS, WARMUP)
sistema(jobs_uniformes, S1_uniforme, S2_uniforme, S3_uniforme)

In [205]:
jobs_exponenciais = gera_jobs(QUANTIDADE_DE_JOBS, WARMUP)
sistema(jobs_exponenciais, S1_exponencial, S2_exponencial, S3_exponencial)

## Métricas

A abordagem adotada para o cálculo das métricas consistiu em determinar o tempo no sistema de cada job como a diferença entre o momento de saída e o momento de chegada. Com base nesses valores, foram calculadas a média do tempo no sistema e o desvio padrão.

In [ ]:
def calcula_metricas(jobs: list[Job]) -> Tuple[float, float]:
  """
  Calcula as métricas de tempo médio no sistema e desvio padrão.
  """
  tempo_total_no_sistema = sum((job.tempo_de_saida - job.tempo_de_chegada) for job in jobs[WARMUP:])
  tempo_medio_no_sistema = tempo_total_no_sistema / QUANTIDADE_DE_JOBS

  desvio_padrao_no_tempo_do_sistema = np.std([job.tempo_de_saida - job.tempo_de_chegada for job in jobs[WARMUP:]])

  return tempo_medio_no_sistema, desvio_padrao_no_tempo_do_sistema

In [ ]:
# efetuando os cálculos para cada situação
metricas_deterministico = calcula_metricas(jobs_deterministicos)
metricas_uniforme = calcula_metricas(jobs_uniformes)
metricas_exponencial = calcula_metricas(jobs_exponenciais)

In [194]:
print(f"Situação 1 (Tempos de serviço determinísticos)\tTempo médio do sistema: {metricas_deterministico[0]}\tDesvio padrão no tempo do sistema: {metricas_deterministico[1]}")
print(f"Situação 2 (Tempos de serviço uniformes)\tTempo médio do sistema: {metricas_uniforme[0]}\tDesvio padrão no tempo do sistema: {metricas_uniforme[1]}")
print(f"Situação 3 (Tempos de serviço exponenciais)\tTempo médio do sistema: {metricas_exponencial[0]}\tDesvio padrão no tempo do sistema: {metricas_exponencial[1]}")

Situação 1 (Tempos de serviço determinísticos)	Tempo médio do sistema: 7.575153236768803	Desvio padrão no tempo do sistema: 8.180536068032259
Situação 2 (Tempos de serviço uniformes)	Tempo médio do sistema: 10079.679878351002	Desvio padrão no tempo do sistema: 10455.74575219189
Situação 3 (Tempos de serviço exponenciais)	Tempo médio do sistema: 6.4722792392945125	Desvio padrão no tempo do sistema: 7.3797113962679575


Os resultados demonstraram como as diferentes distribuições de tempo de serviço impactam o comportamento do sistema. Nas Situações 1 e 2, os tempos médios convergiram para valores entre 5 segundos e 10 segundos, respectivamente, indicando um sistema mais eficiente, com tempos médios reduzidos e variações significativamente menores. Já na Situação 3, onde foi aplicada uma distribuição uniforme para os tempos de serviço, os tempos médios foram consideravelmente mais altos. Esse padrão sugere que essa configuração pode causar um aumento no congestionamento, resultando em um desempenho global menos eficiente.